<div align="center"><h3>HW2, Question 5</h3></div>
<div align="center"><h5>Mohammadreza Ghofrani, 400131076</h5></div>

In [6]:
import os
import glob
import warnings
from itertools import combinations
import cv2
import scipy
import numpy as np
import pandas as pd
import imageio as iio
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import accuracy_score

EPSILON = 0.001
pd.options.plotting.backend = "plotly"

# Question 5

Loading dataset

In [7]:
blood_dataset = pd.read_csv('data/P5/blood_test.txt')

In [8]:
X_COLUMNS = ['mcv', 'alkphos', 'sgpt', 'sgot', 'gammagt']
Y_COLUMN = ['drinks']
train = blood_dataset[blood_dataset['selector'] == 1]
x_train = train[X_COLUMNS]
y_train = train[Y_COLUMN]
test = blood_dataset[blood_dataset['selector'] == 2]
x_test = test[X_COLUMNS]
y_test = test[Y_COLUMN]

## Part A and B

In [9]:
for number_of_samples in [20, 50, 100]:
    train_sample = train.sample(n=number_of_samples)

    cond_prob = {'addicted': None, 'not_addicted': None}

    cond_prob['addicted'] = scipy.stats.multivariate_normal(mean=train_sample[train_sample['drinks'] >= 5][X_COLUMNS].mean(),
                                                            cov=train_sample[train_sample['drinks'] >= 5][X_COLUMNS].cov())
    cond_prob['not_addicted'] = scipy.stats.multivariate_normal(mean=train_sample[train_sample['drinks'] < 5][X_COLUMNS].mean(),
                                                                cov=train_sample[train_sample['drinks'] < 5][X_COLUMNS].cov())

    number_of_test_data = 0
    number_of_misclassified = 0
    for _, t in test.iterrows():
        addicted_likelihood =  cond_prob['addicted'].pdf(x=t[X_COLUMNS]) * EPSILON
        not_addicted_likelihood = cond_prob['not_addicted'].pdf(x=t[X_COLUMNS]) * EPSILON
        prediction = int(addicted_likelihood > not_addicted_likelihood)
        real = int(t[Y_COLUMN][0] >= 5)
        number_of_test_data += 1
        if prediction != real:
            number_of_misclassified += 1
    print(f'Error rate for model created by {number_of_samples} is', number_of_misclassified/number_of_test_data)

Error rate for model created by 20 is 0.405
Error rate for model created by 50 is 0.36
Error rate for model created by 100 is 0.305


## Part C

In [10]:
for number_of_samples in [20, 50, 100]:
    train_sample = train.sample(n=number_of_samples)

    prior_prob = {'addicted': 0, 'not_addicted': 0}
    cond_prob = {'addicted': None, 'not_addicted': None}

    prior_prob['addicted'] = len(train_sample[train_sample['drinks'] >= 5]) / len(train_sample)
    prior_prob['not_addicted'] = 1 - prior_prob['addicted']
    cond_prob['addicted'] = scipy.stats.multivariate_normal(mean=train_sample[train_sample['drinks'] >= 5][X_COLUMNS].mean(),
                                                            cov=train_sample[train_sample['drinks'] >= 5][X_COLUMNS].cov())
    cond_prob['not_addicted'] = scipy.stats.multivariate_normal(mean=train_sample[train_sample['drinks'] < 5][X_COLUMNS].mean(),
                                                                cov=train_sample[train_sample['drinks'] < 5][X_COLUMNS].cov())

    number_of_test_data = 0
    number_of_misclassified = 0
    for _, t in test.iterrows():
        addicted_prob =  prior_prob['addicted'] * cond_prob['addicted'].pdf(x=t[X_COLUMNS]) * EPSILON
        not_addicted_prob = prior_prob['not_addicted'] * cond_prob['not_addicted'].pdf(x=t[X_COLUMNS]) * EPSILON
        prediction = int(addicted_prob > not_addicted_prob)
        real = int(t[Y_COLUMN][0] >= 5)
        number_of_test_data += 1
        if prediction != real:
            number_of_misclassified += 1
    print(f'Error rate for model created by {number_of_samples} is', number_of_misclassified/number_of_test_data)

Error rate for model created by 20 is 0.37
Error rate for model created by 50 is 0.41
Error rate for model created by 100 is 0.31


## Part D

In [11]:
number_of_samples = 100
train_sample = train.sample(n=number_of_samples)
prior_prob = {'addicted': 0, 'not_addicted': 0}
prior_prob['addicted'] = len(train_sample[train_sample['drinks'] >= 5]) / len(train_sample)
prior_prob['not_addicted'] = 1 - prior_prob['addicted']
cond_prob = {'addicted': None, 'not_addicted': None}
max_acc = float("-inf")
best_features = list()
for f1, f2 in combinations(X_COLUMNS, 2):
    cond_prob['addicted'] = scipy.stats.multivariate_normal(mean=train_sample[train_sample['drinks'] >= 5][[f1,f2]].mean(),
                                                        cov=train_sample[train_sample['drinks'] >= 5][[f1, f2]].cov())
    cond_prob['not_addicted'] = scipy.stats.multivariate_normal(mean=train_sample[train_sample['drinks'] < 5][[f1, f2]].mean(),
                                                            cov=train_sample[train_sample['drinks'] < 5][[f1, f2]].cov())

    number_of_misclassified = 0
    y_real = list()
    y_predicted = list()
    for _, t in train_sample.iterrows():
        addicted_prob = cond_prob['addicted'].pdf(x=t[[f1,f2]]) * EPSILON
        not_addicted_prob = cond_prob['not_addicted'].pdf(x=t[[f1,f2]]) * EPSILON
        prediction = int(addicted_prob > not_addicted_prob)
        real = int(t[Y_COLUMN][0] >= 5)
        y_real.append(real)
        y_predicted.append(prediction)
    acc = accuracy_score(y_real, y_predicted)
    if acc > max_acc:
        max_acc = acc
        best_features = [f1 , f2]
f1, f2 = best_features[0], best_features[1]
cond_prob['addicted'] = scipy.stats.multivariate_normal(mean=train_sample[train_sample['drinks'] >= 5][[f1,f2]].mean(),
                                                        cov=train_sample[train_sample['drinks'] >= 5][[f1, f2]].cov())
cond_prob['not_addicted'] = scipy.stats.multivariate_normal(mean=train_sample[train_sample['drinks'] < 5][[f1, f2]].mean(),
                                                        cov=train_sample[train_sample['drinks'] < 5][[f1, f2]].cov())
print(f'features {f1} and {f2} cause maximum accuracy', max_acc)

features sgpt and gammagt cause maximum accuracy 0.8


In [12]:
x = np.linspace(train['mcv'].min() - 10, train['mcv'].max() + 10)
y = np.linspace(train['sgpt'].min() - 10, train['sgpt'].max() + 10)
xx, yy = np.meshgrid(x, y, sparse=False, indexing='ij')

decision_boundary = np.zeros((len(x), len(y)))
for i, xi in enumerate(x):
    for j, yj in enumerate(y):
        decision_boundary[i][j] = int(cond_prob['addicted'].pdf(x=(xi,yj)) > cond_prob['not_addicted'].pdf(x=(xi,yj)))

pos = np.dstack((xx, yy))
addict_countour = cond_prob['addicted'].pdf(pos)
not_addict_countour = cond_prob['not_addicted'].pdf(pos)

fig = go.Figure(data=[
    go.Heatmap(x=x, y=y, z=decision_boundary, showscale=False, name='Decision Boundary'),
    go.Contour(x = x, y =y, z=addict_countour, colorscale='thermal', contours_coloring='lines', line_width=1, showscale=False, name='Addicted Distribution'),
    go.Contour(x = x, y =y, z=not_addict_countour, colorscale='greens', contours_coloring='lines', line_width=1, showscale=False, name='Not Addicted Distribution'),
    ],
    layout={
        'title':{
            'text': f'Distribution and Decision Boundary between {f1} and {f2}',
            'x': 0.5
        },
        'xaxis':{
            'title':{
                'text': f1
            }
        },
        'yaxis': {
            'title': {
                'text': f2
            }
        }
    })
fig.show()

## Part E

In [13]:
x = np.linspace(train['mcv'].min() - 10, train['mcv'].max() + 10)
y = np.linspace(train['sgpt'].min() - 10, train['sgpt'].max() + 10)
xx, yy = np.meshgrid(x, y, sparse=False, indexing='ij')

decision_boundary = np.zeros((len(x), len(y)))
for i, xi in enumerate(x):
    for j, yj in enumerate(y):
        decision_boundary[i][j] = int(3 * prior_prob['addicted'] * cond_prob['addicted'].pdf(x=(xi,yj)) > 2 * prior_prob['not_addicted'] * cond_prob['not_addicted'].pdf(x=(xi,yj)))

pos = np.dstack((xx, yy))
addict_countour = cond_prob['addicted'].pdf(pos)
not_addict_countour = cond_prob['not_addicted'].pdf(pos)

fig = go.Figure(data=[
    go.Heatmap(x=x, y=y, z=decision_boundary, showscale=False, name='Decision Boundary'),
    go.Contour(x = x, y =y, z=addict_countour, colorscale='thermal', contours_coloring='lines', line_width=1, showscale=False, name='Addicted Distribution'),
    go.Contour(x = x, y =y, z=not_addict_countour, colorscale='greens', contours_coloring='lines', line_width=1, showscale=False, name='Not Addicted Distribution'),
    ],
    layout={
        'title':{
            'text': f'Distribution and Decision Boundary between {f1} and {f2} <br> applying lambda12 and lambda21',
            'x': 0.5
        },
        'xaxis':{
            'title':{
                'text': f1
            }
        },
        'yaxis': {
            'title': {
                'text': f2
            }
        }
    })
fig.show()